In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from __future__ import division

from copy import deepcopy
import pickle
import time
import os
import uuid

import scipy
import numpy as np
import h5py

from matplotlib import pyplot as plt
import matplotlib as mpl

from mimi import envs
from mimi import utils
from mimi import user_models
from mimi import opt
from mimi import reward_models
from mimi import models

In [ ]:
data_dir = os.path.join(utils.data_dir, 'x2t')

In [ ]:
sess = utils.make_tf_session(gpu_mode=False)

In [ ]:
env = envs.X2TEnv()

In [ ]:
x2t_dir = os.path.join(data_dir, 'raw')

In [ ]:
def format_eps(obses, actions, rewards):
  eps = []
  for i in range(len(obses)):
    obs = obses[i]
    action = utils.onehot_encode(actions[i], env.n_act_dim)
    reward = rewards[i]
    state = np.zeros(env.n_act_dim)
    state = np.concatenate((state, obs))
    next_state = action
    next_state = np.concatenate((next_state, obs))
    eps.append([(state, action, reward, next_state, True, {})])
  return eps

def load_eps(filename):
  with h5py.File(filename, 'r') as f:
    obses = np.array(f['obses'])
    obses = np.mean(obses, axis=1)
    actions = np.array([f['actions']])[0]
    rewards = np.array(f['rewards'])
  return format_eps(obses, actions, rewards)

In [ ]:
methods = ['default', 'x2t', 'offline']
rollouts_of_pol = []
method_of_pol = []
for method in methods:
  method_path = os.path.join(x2t_dir, method)
  user_ids = os.listdir(method_path)
  for user_id in user_ids:
    user_path = os.path.join(method_path, user_id, 'data.hdf5')
    eps = load_eps(user_path)
    rollouts_of_pol.append(eps)
    method_of_pol.append(method)

In [ ]:
n_conds = len(rollouts_of_pol)
n_steps = sum(len(r) for x in rollouts_of_pol for r in x)
n_steps, n_conds, n_steps / n_conds

In [ ]:
mi_model_init_args = [sess]
mi_model_init_kwargs = {
  'n_env_obs_dim': env.n_env_obs_dim,
  'n_user_obs_dim': env.n_user_obs_dim,
  'n_act_dim': env.n_act_dim,
  'n_layers': 2,
  'layer_size': 64
}
mi_model_train_kwargs = {
  'iterations': 1000,
  'ftol': 1e-6,
  'learning_rate': 1e-4,
  'batch_size': 64,
  'val_update_freq': None,
  'verbose': False,
  'warm_start': False
}

In [ ]:
reward_model = reward_models.MIRewardModel(
  env,
  mi_model_init_args,
  mi_model_init_kwargs,
  mi_model_train_kwargs
)

In [ ]:
ixs_reward_model = reward_models.MIRewardModel(
  env,
  mi_model_init_args,
  mi_model_init_kwargs,
  mi_model_train_kwargs,
  use_next_env_obs=False
)

In [ ]:
true_rew_of_rollout = lambda rollout: np.mean([x[2] for x in rollout])
true_reward_model = lambda rollouts: np.mean([true_rew_of_rollout(rollout) for rollout in rollouts])

In [ ]:
offline_reward_models = [true_reward_model, reward_model, ixs_reward_model]

In [ ]:
n_seeds = 10
rewards_of_pol = np.zeros((n_seeds, len(rollouts_of_pol), len(offline_reward_models)))

In [ ]:
for i in range(n_seeds):
  rewards_of_pol[i, :, :] = utils.compute_rews_of_rollouts(
    rollouts_of_pol,
    offline_reward_models,
    verbose=True
  )

In [ ]:
rewards_of_pol_path = os.path.join(data_dir, 'rewards_of_pol.pkl')

In [ ]:
with open(rewards_of_pol_path, 'wb') as f:
  pickle.dump(rewards_of_pol, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(rewards_of_pol_path, 'rb') as f:
  rewards_of_pol = pickle.load(f)

In [ ]:
rewards_of_pol[:, :, 1:] = np.maximum(rewards_of_pol[:, :, 1:], 0)

In [ ]:
mean_rewards_of_pol = np.mean(rewards_of_pol, axis=0)
mean_rewards_of_pol

In [ ]:
mean_rewards_of_pol[:, 2] = np.minimum(mean_rewards_of_pol[:, 2], mean_rewards_of_pol[:, 1])
mean_rewards_of_pol[:, 2] = np.maximum(mean_rewards_of_pol[:, 2], 0)
mean_rewards_of_pol = np.concatenate((mean_rewards_of_pol, (mean_rewards_of_pol[:, 1] - mean_rewards_of_pol[:, 2])[:, np.newaxis]), axis=1)
mean_rewards_of_pol

In [ ]:
color_of_method = {
  'default': 'gray',
  'offline': 'teal',
  'x2t': 'orange'
}
label_of_method = {
  'default': 'Non-Adaptive (After X2T)',
  'offline': 'Non-Adaptive (Before X2T)',
  'x2t': 'X2T'
}
color_of_pol = [color_of_method[m] for m in method_of_pol]

In [ ]:
idxes_of_method = {m: [] for m in methods}
for i, method in enumerate(method_of_pol):
  idxes_of_method[method].append(i)

In [ ]:
mpl.rcParams.update({'font.size': 12})

In [ ]:
corr = scipy.stats.spearmanr(mean_rewards_of_pol)
corr

In [ ]:
rho = corr[0][0, 1]
rho

In [ ]:
plt.title(r'X2T ($\rho$ = %0.2f)' % rho)
plt.xlabel('True Reward (Classification Accuracy)')
plt.ylabel(r"$\mathcal{I}(\mathbf{x}_t, (\mathbf{s}_t, \mathbf{s}_{t+1}))$")
for method, idxes in idxes_of_method.items():
  plt.scatter(
    mean_rewards_of_pol[idxes, 0], 
    mean_rewards_of_pol[idxes, 1],
    color=color_of_method[method],
    label=label_of_method[method],
    s=50
  )
plt.legend(loc='best')
plt.savefig(os.path.join(data_dir, 'x2t-offline-eval-truerew-vs-mi-per-poluser.pdf'), bbox_inches='tight')
plt.show()

In [ ]:
rewards_of_method = [[[] for _ in range(mean_rewards_of_pol.shape[1])] for _ in methods]
idx_of_method = {x: i for i, x in enumerate(methods)}
for i, method in enumerate(method_of_pol):
  for j in range(mean_rewards_of_pol.shape[1]):
    rewards_of_method[idx_of_method[method]][j].append(mean_rewards_of_pol[i, j])
rewards_of_method = [[np.mean(x) for x in y] for y in rewards_of_method]
rewards_of_method = np.array(rewards_of_method)

In [ ]:
plt.title('X2T')
plt.xlabel('True Reward (Classification Accuracy)')
plt.ylabel(r"$\mathcal{I}(\mathbf{x}_t, (\mathbf{s}_t, \mathbf{s}_{t+1}))$")
for i, method in enumerate(methods):
  plt.scatter(
    rewards_of_method[i, 0], 
    rewards_of_method[i, 1],
    color=color_of_method[method],
    label=label_of_method[method],
    s=50
  )
plt.legend(loc='lower right')
plt.savefig(os.path.join(data_dir, 'x2t-offline-eval-truerew-vs-mi-per-pol.pdf'), bbox_inches='tight')
plt.show()